In [1]:
# Import libs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv("D:/Datasets/Churn_Modelling.csv")

# Veri keşfi

In [4]:
df.head()

RowNumber  CustomerId   Surname  CreditScore Geography  Gender  Age  \
0          1    15634602  Hargrave          619    France  Female   42   
1          2    15647311      Hill          608     Spain  Female   41   
2          3    15619304      Onio          502    France  Female   42   
3          4    15701354      Boni          699    France  Female   39   
4          5    15737888  Mitchell          850     Spain  Female   43   

   Tenure    Balance  NumOfProducts  HasCrCard  IsActiveMember  \
0       2       0.00              1          1               1   
1       1   83807.86              1          0               1   
2       8  159660.80              3          1               0   
3       1       0.00              2          0               0   
4       2  125510.82              1          1               1   

   EstimatedSalary  Exited  
0        101348.88       1  
1        112542.58       0  
2        113931.57       1  
3         93826.63       0  
4         79084.10       0

In [5]:
len(df.columns)

14

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             10000 non-null int64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [8]:
df.groupby(['Exited']).RowNumber.agg(['count'])

count
Exited       
0        7963
1        2037

In [9]:
# Bağımlı ve bağımsız değişkenleri ayırma
# RowNumber, CustomerId, Surname hedef değişkeni tahmin etme bakımından işe yaramaz. Bu yüzden 0,1,2 indeksli nitelikleri
# Nitelikler matrisine dahil etmiyoruz. Yukarıda kaç tane nitelik olduğunu yazdırmıştık. Şimdi 0,1,2 hari ise 3'ten başlayacak. 
# Python'da indeks değeri 0'dan başladığına ve 14 nitalik olduğuna göre son niteliğin indeks değeri 13 olacaktır. iloc ile sütun
#seçerken ilk değer dahil ikinci değer hariçtir. bu sebeple 3'ü dahil 13'ü hariç tutacağız. Çünkü son indeks değeri Exited yani
# hedef değişkene geliyor. İlk : tüm satırlar "," sonrası sütunlardan seçilecek filtre. 3:13 demek 3 dahil 13 hariç indeksli
# sütunları seç demek

X = df.iloc[:,3:13].values

# Hedef niteliğimiz 13. indekste idi. O sebeple : ile tüm satırları "," den sonra 13 ile hedef niteliğin indeksini seçiyoruz.
y = df.iloc[:,13].values

In [10]:
X.shape

(10000, 10)

In [11]:
y.shape

(10000,)

# Kategorik Nitelikler için LabelEncoder

In [12]:
# Hedef nitelik 0 ve 1 olmak üzere zaten nümerik değer olduğu için onu LabelEncoder ile kodlamaya gerek yok.
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# X nitelikler matrisini incelediğimizde 1'inci indekste Geography 2'nci indekste Gender'ın kategorik olduğunu görüyoruz.
# Önce Geagraphy için LabelEncoder yaratalım
labelEncoder_Geo = LabelEncoder()
X[:, 1] = labelEncoder_Geo.fit_transform(X[:, 1])

In [13]:
# Şimdi Gender için
labelEncoder_Gender = LabelEncoder()
X[:,2] = labelEncoder_Gender.fit_transform(X[:,2])

# OneHotEncoder

In [14]:
from sklearn.preprocessing import OneHotEncoder
onehotencoder = OneHotEncoder(handle_unknown='ignore', categories='auto')

In [15]:
# Birden sütunu ilgilendiren dönüşümleri kolay yapabilmek için kullanışlı bir sınıf (Estimator kökenli)
# Biz burada sadece OneHotEncoder için kullanacağız.
from sklearn.compose import ColumnTransformer

# one_hot_encode_with_ct bizim kendi verdiğiiz isimdir. onehotencoder'ı yukarıda yarattık. [1] ise dönüşecek
# niteliklerin indeks değerleridir (Geography) Gender binary olduğu için sokmaya gerek yok.
ct = ColumnTransformer(
    [('one_hot_encode_with_ct', onehotencoder, [1]),], 
    remainder='passthrough'
)

In [16]:
# ColumnTransformer nesnesi ile onehotencoder nesnesini eğitip gerekli dönüşümleri gerçekleştirip X'e tekrar atayalım.
# Burada X'in boyutu(shape) değişecektir.
X = ct.fit_transform(X)

In [17]:
X.shape

(10000, 12)

In [18]:
X[:3]

array([[1.0, 0.0, 0.0, 619, 0, 42, 2, 0.0, 1, 1, 1, 101348.88],
       [0.0, 0.0, 1.0, 608, 0, 41, 1, 83807.86, 1, 0, 1, 112542.58],
       [1.0, 0.0, 0.0, 502, 0, 42, 8, 159660.8, 3, 1, 0, 113931.57]],
      dtype=object)

In [19]:
# Gölge Değişken Tuzağından kurtulmak için Yeni oluşturulan gölge değişkenlerden birini çıkaralım (0 çıkıyor)
X = X[:,1:]

In [20]:
X.shape

(10000, 11)

# Feature Scaling

In [21]:
# Nitelikler matrisini (X) StandardScaler
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X = scaler.fit_transform(X)

In [22]:
X[:3]

array([[-0.57873591, -0.57380915, -0.32622142, -1.09598752,  0.29351742,
        -1.04175968, -1.22584767, -0.91158349,  0.64609167,  0.97024255,
         0.02188649],
       [-0.57873591,  1.74273971, -0.44003595, -1.09598752,  0.19816383,
        -1.38753759,  0.11735002, -0.91158349, -1.54776799,  0.97024255,
         0.21653375],
       [-0.57873591, -0.57380915, -1.53679418, -1.09598752,  0.29351742,
         1.03290776,  1.33305335,  2.52705662,  0.64609167, -1.03067011,
         0.2406869 ]])

# Eğitim ve Test Seti olarak Ayırmak

In [23]:
from sklearn.model_selection import train_test_split

#Sıralamaya dikkat. Aynı sonuçlar için random_state değeri aynı olmalıdır.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=142)

# Bölüm 2 ANN

In [22]:
# Bu bölümden önce aşağıdaki kütüphaneler kurulmalıdır.
# !pip install theano

# !pip install tensorflow

# !pip install keras


In [24]:
import tensorflow as tf

In [25]:
tf.__version__

'1.14.0'

In [26]:
# Keras kütüphanelerini indirme
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


## YAPAY SİNİR AĞI NESNESİ OLUŞTURMA

In [27]:
# Yapay sinir ağı iki farklı şekilde başlatılabilir: 
# 1. Katmanlar dizilimi (Sequantial layers) olarak. 
# 2. Graph olarak.
# Birinci yolda sadece yukarıda indirilen Sequantial sınıfından nesne yaratılır. 
# Bu da Karar ağacı, SVM gibi bir sınıflandırıcı olacak.
classifier = Sequential()

W1117 11:47:44.513602 10004 deprecation_wrapper.py:119] From c:\users\user\python_egitim\tensorflow-1\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



## Bir Sinir Ağını Stochastic Gradient Descent ile Eğitmenin Aşamaları

    Sıfıra yakın rastgele değerler ile ağırlık katsayılarını belirle.
    Input layera ilk gözlemi(bir satır) her bir düğüme bir nitelik (nitelikler matrisi sütün sayısı kadar) düşecek şekilde ver.
    Ağı çalıştır ve ilk tahmin y değerini üret.
    Üretilen y ile tahmin edilen y arasındaki hatayı hesapla.
    Hesaplanan hatayı geri besleme olarak gönder ve her bir ağırlık katsayısını güncelle.
    1 ve 5 arasını ya her bir satırdan sonra veya belli bir satır sayısından (grup-batch) sonra tekrarla.
    Veri setindeki tüm satırların bitmesi bir epoch oluşturur. Bunu defalarca tekrarla.

## GİRDİ VE İLK GİZLİ KATMANI EKLEME

In [28]:
# classifier nesnesinin add() metodu içine Dense() Sınıfının nesnesnesini parametre olarak veriyoruz.
# Dense içindeki ilk parametre 
# kernel_initializer (eski init): Başlangıç ağırlıklarını belirler. uniform fonksiyonunu kullanıyoruz.
# input_dim: Yapay sinir ağı nesnesine ilk katman ekleme işlemi yapılırken mutlaka girdi katmanında kaç düğüm olacağı bildirilir.
# aksi halde ilk gizli katman kendisine kaç düğümden bağlantı olacak bilemez. 
# Bu rakam nitelikler matrisindeki sütun sayısıdır yani 11
# units (eski:output_dim): Bu sayının nasıl verileceğine dair sağlam bir kural yok. Biraz sanatkarlık ve tecrübe
# gerektiriyor. Tecrübeye dayalı ipucu: Girdi ve çıktı düğüm sayısının ortalamasını verebilirsin. 
# Bu örnekte (11+1=12/2=6) Ya da parametre tuning tekniklerini kullanabilirsin.
# activation: Gizli düğümler için rectifier çıktı düğüm için sigmoid olsun. Rectifier action parametresinde relu olarak tanımlı


classifier.add(Dense(kernel_initializer = tf.random.uniform, 
                     input_dim = 11, 
                     units = 6,  
                     activation = 'relu'))

W1117 11:48:57.901925 10004 deprecation_wrapper.py:119] From c:\users\user\python_egitim\tensorflow-1\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



## İKİNCİ GİZLİ KATMANI EKLEME

In [29]:
# İkinci Gizli Katmanı Eklemek

# Yukarıda ilk gizli katman ve düğüm sayısı belli olduğu için burada input_dim parametresi kullanmaya gerek yok.
# Gizli katman sayısını (units) aynı tutuyoruz. kaba formül kullandık yine.
classifier.add(Dense(kernel_initializer = tf.random.uniform, units = 4,  activation = 'relu'))

## ÇIKTI KATMANI (OUTPUT LAYER) EKLEME

In [30]:
# Bu katmanın bir öncekinden farkı düğüm sayısı ve activasyon fonksiyonu olacak

classifier.add(Dense(kernel_initializer = tf.random.uniform, units = 1,  activation = 'sigmoid'))

## YAPAY SINIR AĞINI DERLEME (COMPILE ANN)

In [31]:
! pip install pillow

Traceback (most recent call last):
  File "c:\program files\python36\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\program files\python36\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Program Files\Python36\Scripts\pip.exe\__main__.py", line 9, in <module>
TypeError: 'module' object is not callable


In [32]:
# optimizer: Stochastic Gradient Descent'i temsilen adam
# loss: SGD'ni optimizasyonu için kullanılacak loss function. 
#       Tahmin y ile gerçek ye değeri arasını hesaplayıp en optimal değeri
#        SGD'ye buldurur. Logaritmik loss function. Binary olduğu için 
#       binary_crossentropy kullanıyoruz.
# metrics: İlave olarak burada model değerlendirme kriterleri belirlenir. Bir liste halinde verilir. 
#          Biz şimdilik sadece accuracy veriyoruz.


classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

W1117 11:50:50.974818 10004 deprecation_wrapper.py:119] From c:\users\user\python_egitim\tensorflow-1\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W1117 11:50:50.993767 10004 deprecation_wrapper.py:119] From c:\users\user\python_egitim\tensorflow-1\lib\site-packages\keras\backend\tensorflow_backend.py:3376: The name tf.log is deprecated. Please use tf.math.log instead.

W1117 11:50:50.999752 10004 deprecation.py:323] From c:\users\user\python_egitim\tensorflow-1\lib\site-packages\tensorflow\python\ops\nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


# Yapay Sinir Ağını Eğitim Verisi ile Eğitmek

In [33]:
classifier.fit(X_train, y_train, batch_size=5, epochs=20)

W1117 11:51:51.024353 10004 deprecation_wrapper.py:119] From c:\users\user\python_egitim\tensorflow-1\lib\site-packages\keras\backend\tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W1117 11:51:51.131064 10004 deprecation_wrapper.py:119] From c:\users\user\python_egitim\tensorflow-1\lib\site-packages\keras\backend\tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.



Epoch 1/20
8000/8000 [==============================] - 1s 175us/step - loss: 1.0133 - acc: 0.6690
Epoch 2/20
8000/8000 [==============================] - 1s 145us/step - loss: 0.5071 - acc: 0.7950
Epoch 3/20
8000/8000 [==============================] - 1s 146us/step - loss: 0.4756 - acc: 0.8016
Epoch 4/20
8000/8000 [==============================] - 1s 146us/step - loss: 0.4408 - acc: 0.8118
Epoch 5/20
8000/8000 [==============================] - 1s 148us/step - loss: 0.4122 - acc: 0.8259
Epoch 6/20
8000/8000 [==============================] - 1s 147us/step - loss: 0.3907 - acc: 0.8365
Epoch 7/20
8000/8000 [==============================] - 1s 148us/step - loss: 0.3783 - acc: 0.8446
Epoch 8/20
8000/8000 [==============================] - 1s 145us/step - loss: 0.3715 - acc: 0.8475
Epoch 9/20
8000/8000 [==============================] - 1s 144us/step - loss: 0.3661 - acc: 0.8491
Epoch 10/20
8000/8000 [==============================] - 1s 144us/step - loss: 0.3624 - acc: 0.8488
Epoch 11/

# Yapay Sinir Ağını Test verisi ile Test Etmek

In [34]:
y_pred = classifier.predict(X_test)

In [35]:
y_pred[:4]

array([[0.44946533],
       [0.9060261 ],
       [0.4672837 ],
       [0.06024021]], dtype=float32)

In [36]:
# y_pred için olasılık değerlerini görüyoruz. 0.5'ten büyük olanları 1 olarak kabul edelim.
y_pred = [1 if i > 0.5 else 0 for i in y_pred]

In [37]:
y_pred[:4]

[0, 1, 0, 0]

In [38]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[1565,   65],
       [ 199,  171]], dtype=int64)

In [39]:
from sklearn.metrics import accuracy_score

In [40]:
accuracy_score(y_pred=y_pred, y_true=y_test)

0.868